# Ανάκτηση Πληροφορίας

# Lab Project 2024-2025: Δημιουργία Μηχανής Αναζήτησης

# Αναστασόπουλος Ευθύμιος - ΑΜ 21390292

## Περιγραφή Εργασίας
Στην εργασία αυτή, αναπτύσσουμε μια βασική μηχανή αναζήτησης με στόχο την κατανόηση και εφαρμογή των βασικών εννοιών της ανάκτησης πληροφορίας, όπως η συλλογή δεδομένων, η επεξεργασία κειμένου, η δημιουργία ευρετηρίου και η κατάταξη αποτελεσμάτων. Το έργο ακολουθεί τα εξής βήματα:

Συλλογή δεδομένων: Χρήση ενός crawler για την άντληση άρθρων από το Wikipedia.
Προεπεξεργασία δεδομένων: Καθαρισμός και προετοιμασία των δεδομένων κειμένου για ανάλυση.
Δημιουργία ευρετηρίου: Κατασκευή ανεστραμμένου ευρετηρίου για γρήγορη αναζήτηση.
Μηχανή αναζήτησης: Υλοποίηση αλγορίθμων για την ανάκτηση σχετικών άρθρων βάσει ερωτημάτων.
Αξιολόγηση: Μέτρηση της αποτελεσματικότητας της μηχανής αναζήτησης με κατάλληλες μετρικές.
Η εργασία υλοποιείται με τη χρήση Python και συνοδεύεται από ένα αναλυτικό Jupyter Notebook που περιλαμβάνει παραδείγματα και σχόλια για κάθε βήμα.

---
## Βήμα 1: Συλλογή Δεδομένων

**Σκοπός:** Συλλογή άρθρων από το Wikipedia μέσο web crawler, τα οποία θα αποτελέσουν το dataset για τη μηχανή αναζήτησης.

**Περιγραφή:**
Στο πρώτο βήμα, συλλέγουμε δεδομένα από το Wikipedia με στόχο τη δημιουργία ενός συνόλου κειμένων που θα χρησιμοποιηθεί στα επόμενα στάδια της μηχανής αναζήτησης. Η διαδικασία περιλαμβάνει τα εξής βήματα:

1. Ορισμός Ερωτήματος Αναζήτησης: Επιλέγουμε ένα query (στην περίπτωση αυτή, "Social Media Entertainment") για να αντλήσουμε σχετικά άρθρα.
2. Χρήση Crawler: Αναπτύσσουμε και χρησιμοποιούμε έναν web crawler που επισκέπτεται τη σελίδα αναζήτησης του Wikipedia, συλλέγει τους συνδέσμους των άρθρων και εξάγει τον τίτλο, το URL και το περιεχόμενο κάθε άρθρου.
3. Περιορισμός Αποτελεσμάτων: Ορίζουμε τον αριθμό των άρθρων που θα συλλεχθούν (15 άρθρα).
4. Αποθήκευση Δεδομένων: Αποθηκεύουμε τα άρθρα σε αρχείο wikipedia_articles.json σε δομημένη μορφή, περιλαμβάνοντας τον τίτλο, το URL και το περιεχόμενο κάθε άρθρου.


# Υλοποίηση `crawler.py`

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def fetch_wikipedia_articles(query, num_articles=15):
    # Βασικά URLs
    search_url = f"https://en.wikipedia.org/w/index.php?search={query}"
    
    # Αναζήτηση στο Wikipedia
    response = requests.get(search_url)
    if response.status_code != 200:
        print("Σφάλμα κατά την ανάκτηση της σελίδας αναζήτησης.")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')

    # Εύρεση συνδέσμων άρθρων
    links = [a['href'] for a in soup.select('a[href^="/wiki/"]') if ':' not in a['href']][:num_articles]

    articles = []
    for link in links:
        url = f"https://en.wikipedia.org{link}"
        article_response = requests.get(url)
        if article_response.status_code != 200:
            print(f"Σφάλμα κατά την ανάκτηση του άρθρου: {url}")
            continue
        
        article_soup = BeautifulSoup(article_response.text, 'html.parser')

        # Εξαγωγή τίτλου και περιεχομένου
        title = article_soup.find('h1').text
        paragraphs = article_soup.find_all('p')
        content = ' '.join([p.text.strip() for p in paragraphs])

        # Προσθήκη του άρθρου
        articles.append({
            "title": title,
            "url": url,
            "content": content
        })

    return articles

def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Τα άρθρα αποθηκεύτηκαν στο αρχείο: {filename}")

if __name__ == "__main__":
    query = "Social Media Entertainment"  # Ερώτημα αναζήτησης
    num_articles = 15  # Αριθμός άρθρων
    articles = fetch_wikipedia_articles(query, num_articles)

    if articles:
        save_to_json(articles, "wikipedia_articles.json")


In [4]:
import json

file_path = "/home/makis/Documents/GitHub/irlabproject/data/wikipedia_articles.json"

with open(file_path, "r", encoding="utf-8") as f:
    wikipedia_data = json.load(f)

print(json.dumps(wikipedia_data[:2], ensure_ascii=False, indent=4))


[
    {
        "title": "Main Page",
        "url": "https://en.wikipedia.org/wiki/Main_Page",
        "content": "Volcanism of the Mount Edziza volcanic complex in British Columbia, Canada, spans more than 7 million years. The first magmatic cycle took place between 7.5 and 6 million years ago and is represented by the Raspberry, Little Iskut and Armadillo geological formations. Volcanism has taken place during five cycles of magmatic activity, each producing less volcanic material than the previous one. During these cycles volcanism has created several types of volcanoes, including cinder cones, stratovolcanoes, subglacial volcanoes, shield volcanoes and lava domes. The roughly 1,000-square-kilometre (400-square-mile) volcanic plateau of the MEVC originated from the successive eruptions of highly mobile lava flows. Several types of volcanic rocks were deposited by multiple eruptions of the MEVC. At least 10 distinct flows of obsidian were produced by volcanism of the MEVC, some of w

# Αποτελέσματα βήματος 1:

# Δημιουργία αρχείου json αποθήευσης αρχείων (dataset) μέσω `crawler.py:` `wikipedia_articles.json`
To `wikipedia_articles.json` αρχείο περιέχει 15 άρθρα και το περιεχόμενο τους χωρίζεται σε τίτλο, URL και περιεχόμενο.

---
# Βήμα 2: Προεπεξεργασία κειμένου (Text Processing)

**Σκοπός:** Στο δεύτερο βήμα, πραγματοποιούμε την προεπεξεργασία των δεδομένων που συλλέξαμε από το Wikipedia, ώστε να τα προετοιμάσουμε για τη δημιουργία ευρετηρίου. Η διαδικασία περιλαμβάνει τα εξής:

**1. Φόρτωση των δεδομένων:**

Διαβάζουμε το αρχείο wikipedia_articles.json που δημιουργήθηκε στο Βήμα 1 και περιλαμβάνει τα άρθρα από το Wikipedia.

**2.Καθαρισμός κειμένου:**

Αφαίρεση ειδικών χαρακτήρων, αριθμών και σημείων στίξης.

Μετατροπή όλων των λέξεων σε πεζά για ομοιογένεια.

**3. Tokenization:**

Διαχωρισμός του κειμένου σε ξεχωριστές λέξεις (tokens).

**4. Αφαίρεση Stopwords:**

Αφαιρούμε κοινές λέξεις (π.χ., "και", "ο", "αυτός") που δεν συμβάλλουν στη σημασιολογική ανάλυση.

**5.Stemming ή Lemmatization:**

Μειώνουμε τις λέξεις στη ρίζα τους (π.χ., "τρέχει", "τρέξιμο" → "τρέχ").

**6. Αποθήκευση των καθαρισμένων δεδομένων:**

Τα δεδομένα αποθηκεύονται σε ένα νέο αρχείο cleaned_wikipedia_articles.json για χρήση στα επόμενα βήματα.

# Υλοποίηση `preprocessing.py`


In [ ]:
import json
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk

# Κατέβασμα απαραίτητων δεδομένων του NLTK
nltk.download('punkt')
nltk.download('stopwords')

# Αρχικοποίηση των εργαλείων
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Συνάρτηση για καθαρισμό και επεξεργασία κειμένου
def preprocess_text(text):
    
    # Μετατροπή σε πεζά
    text = text.lower()
    # Αφαίρεση ειδικών χαρακτήρων και αριθμών
    text = re.sub(r"[^a-z\s]", "", text)
    # Tokenization
    tokens = word_tokenize(text)
    # Αφαίρεση stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    return " ".join(stemmed_tokens)

# Φόρτωση των δεδομένων από το JSON αρχείο
def load_articles(file_path):

    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

# Αποθήκευση δεδομένων σε JSON αρχείο
def save_articles(articles, file_path):

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(articles, f, ensure_ascii=False, indent=4)

# Κύρια συνάρτηση προεπεξεργασίας
def preprocess_articles(input_file, output_file):

    articles = load_articles(input_file)
    for article in articles:
        article["processed_content"] = preprocess_text(article["content"])
    save_articles(articles, output_file)
    print(f"Η προεπεξεργασία ολοκληρώθηκε! Τα δεδομένα αποθηκεύτηκαν στο {output_file}")

if __name__ == "__main__":
    input_file = "data/wikipedia_articles.json"
    output_file = "data/cleaned_wikipedia_articles.json"
    preprocess_articles(input_file, output_file)


## Αποτελέσματα
Σε αυτό το βήμα, καθαρίσαμε το περιεχόμενο των άρθρων που συλλέξαμε, ώστε να είναι έτοιμα για ευρετηρίαση. 
Η διαδικασία περιλαμβάνει μετατροπή κεφαλαίων σε πεζά, tokeninzation, αφαίρεση ειδικών χαρακτήρων, αφαίρεση stopwords, και stemming. 
Τα δεδομένα αποθηκεύονται σε νέο αρχείο `cleaned_wikipedia_articles.json`.

In [ ]:
import json

file_path = "/home/makis/Documents/GitHub/irlabproject/data/cleaned_wikipedia_articles.json"

with open(file_path, "r", encoding="utf-8") as f:
    cleaned_data = json.load(f)

print(json.dumps(cleaned_data[:2], ensure_ascii=False, indent=4))


[
    {
        "title": "Main Page",
        "url": "https://en.wikipedia.org/wiki/Main_Page",
        "content": "Volcanism of the Mount Edziza volcanic complex in British Columbia, Canada, spans more than 7 million years. The first magmatic cycle took place between 7.5 and 6 million years ago and is represented by the Raspberry, Little Iskut and Armadillo geological formations. Volcanism has taken place during five cycles of magmatic activity, each producing less volcanic material than the previous one. During these cycles volcanism has created several types of volcanoes, including cinder cones, stratovolcanoes, subglacial volcanoes, shield volcanoes and lava domes. The roughly 1,000-square-kilometre (400-square-mile) volcanic plateau of the MEVC originated from the successive eruptions of highly mobile lava flows. Several types of volcanic rocks were deposited by multiple eruptions of the MEVC. At least 10 distinct flows of obsidian were produced by volcanism of the MEVC, some of w

---

# Βήμα 3: Δημιουργία Ευρετηρίου (Indexing)
**Σκοπός:** Σε αυτό το βήμα, δημιουργούμε ένα ανεστραμμένο ευρετήριο που συνδέει κάθε λέξη με τη λίστα άρθρων στα οποία εμφανίζεται.

**1. Φόρτωση καθαρισμένων δεδομένων:**

Χρησιμοποιούμε το cleaned_wikipedia_articles.json που δημιουργήθηκε στο Βήμα 2.

**2. Δημιουργία Ανεστραμμένου Ευρετηρίου:**

Για κάθε λέξη (token) στα δεδομένα:
Προσθέτουμε τη λέξη ως κλειδί στο ευρετήριο.
Αποθηκεύουμε τα άρθρα στα οποία εμφανίζεται η λέξη.

**3. Αποθήκευση Ευρετηρίου:**

Το ανεστραμμένο ευρετήριο αποθηκεύεται σε αρχείο inverted_index.json για χρήση στη μηχανή αναζήτησης.

**4. Βελτιώσεις:**

Αφαίρεση κοινών λέξεων (π.χ. stopwords) και λέξεων με λίγες εμφανίσεις (π.χ. 1-2).
Χρήση δομής δεδομένων για ταχύτερη αναζήτηση.

## Υλοποίηση `indexing.py`

In [ ]:
import json
from collections import defaultdict

# Φόρτωση των καθαρισμένων δεδομένων
def load_cleaned_articles(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

# Δημιουργία ανεστραμμένου ευρετηρίου
def create_inverted_index(articles):
    inverted_index = defaultdict(list)
    
    for article_id, article in enumerate(articles):
        tokens = article["processed_content"].split()
        for token in set(tokens):  # Χρησιμοποιούμε set για μοναδικότητα λέξεων
            inverted_index[token].append(article_id)
    
    return inverted_index

# Αποθήκευση ανεστραμμένου ευρετηρίου
def save_inverted_index(inverted_index, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(inverted_index, f, ensure_ascii=False, indent=4)

# Κύρια συνάρτηση
def build_index(input_file, output_file):
    articles = load_cleaned_articles(input_file)
    inverted_index = create_inverted_index(articles)
    save_inverted_index(inverted_index, output_file)
    print(f"Το ανεστραμμένο ευρετήριο αποθηκεύτηκε στο {output_file}")

if __name__ == "__main__":
    input_file = "data/cleaned_wikipedia_articles.json"
    output_file = "data/inverted_index.json"
    build_index(input_file, output_file)


# Αποτελέσματα ευρετηρίου

In [ ]:
import json

file_path = "/home/makis/Documents/GitHub/irlabproject/data/inverted_index.json"

with open(file_path, "r", encoding="utf-8") as f:
    inverted_index = json.load(f)

sample = {key: inverted_index[key] for key in list(inverted_index.keys())[:5]}
print(json.dumps(sample, ensure_ascii=False, indent=4))


---

# Βήμα 4: Μηχανή Αναζήτησης (Search Engine)

Στο τέταρτο βήμα, αναπτύσσουμε μια μηχανή αναζήτησης που επιτρέπει στους χρήστες να αναζητούν έγγραφα με βάση λέξεις-κλειδιά (terms) και να λαμβάνουν σχετικά αποτελέσματα, ταξινομημένα κατά συνάφεια.

#### Υποσυστήματα:

1. **Επεξεργασία Ερωτήματος (Query Processing):**
   Το σύστημα λαμβάνει ερωτήματα από τους χρήστες.
   Τα ερωτήματα υποβάλλονται σε προεπεξεργασία (tokenization, αφαίρεση stopwords, stemming).
   Υποστηρίζονται λογικές λειτουργίες Boolean:
     **AND**: Επιστρέφονται έγγραφα που περιέχουν όλους τους όρους.
     **OR**: Επιστρέφονται έγγραφα που περιέχουν οποιονδήποτε από τους όρους.
     **NOT**: Εξαιρούνται έγγραφα που περιέχουν συγκεκριμένους όρους.

2. **Κατάταξη Αποτελεσμάτων (Ranking):**
   Εφαρμόζουμε πολλαπλούς αλγόριθμους ανάκτησης:
     **Boolean Retrieval**: Βασική αναζήτηση με λογικές πράξεις.
     **Vector Space Model (VSM)**: Υπολογισμός cosine similarity μεταξύ ερωτήματος και εγγράφων.
     **Okapi BM25**: Προηγμένος αλγόριθμος κατάταξης βασισμένος στη συχνότητα όρων.

3. **Διεπαφή Χρήστη (User Interface):**
   Ο χρήστης εισάγει ένα ερώτημα και επιλέγει μέθοδο ανάκτησης.
   Τα αποτελέσματα εμφανίζονται ταξινομημένα κατά συνάφεια.

#### Ορόσημα:
**Επεξεργαστής Ερωτημάτων**: Ικανός να χειρίζεται Boolean ερωτήματα.
**Κατάταξη Αποτελεσμάτων**: Μια βελτιωμένη μηχανή αναζήτησης με ταξινομημένα αποτελέσματα βάσει πολλαπλών αλγορίθμων.

#### Αποτελέσματα:
Τα αποτελέσματα εμφανίζονται με τη μορφή:
**Τίτλος**: Τίτλος του εγγράφου.
**Συνάφεια**: Βαθμολογία που υπολογίζεται από τον επιλεγμένο αλγόριθμο.
**Περιεχόμενο**: Απόσπασμα του εγγράφου.

## Παρακάτω παρουσιάζεται η υλοποίηση της μηχανής αναζήτησης `search_engine.py`.


In [ ]:
import json
import math
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from collections import Counter
import numpy as np
from rank_bm25 import BM25Okapi
import nltk

# Κατέβασμα των απαραίτητων δεδομένων
nltk.download('punkt')
nltk.download('stopwords')

# Φόρτωση δεδομένων
def load_data():
    with open('data/inverted_index.json', 'r', encoding='utf-8') as file:
        inverted_index = json.load(file)

    with open('data/cleaned_wikipedia_articles.json', 'r', encoding='utf-8') as file:
        processed_articles = json.load(file)
    
    return inverted_index, processed_articles

# Προεπεξεργασία ερωτήματος
def preprocess_query(query):
    query = query.lower()
    query = re.sub(r'[^a-z\s]', '', query)
    tokens = word_tokenize(query)
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

# Boolean αναζήτηση με υποστήριξη AND, OR, NOT
def boolean_search(query, index, processed_articles, operation="AND"):
    tokens = preprocess_query(query)
    print(f"Query Tokens: {tokens}")
    
    if not tokens:
        return set()

    if operation == "AND":
        results = set(index.get(tokens[0], []))
        for token in tokens[1:]:
            results = results.intersection(set(index.get(token, [])))
    elif operation == "OR":
        results = set()
        for token in tokens:
            results = results.union(set(index.get(token, [])))
    elif operation == "NOT":
        results = set(range(len(processed_articles)))
        for token in tokens:
            results = results.difference(set(index.get(token, [])))
    else:
        raise ValueError("Unsupported operation. Use 'AND', 'OR', or 'NOT'.")
    
    return results

# Υπολογισμός cosine similarity για VSM
def vsm_search(query, index, documents):
    query_tokens = preprocess_query(query)
    print(f"VSM Query Tokens: {query_tokens}")

    # Υπολογισμός IDF
    N = len(documents)
    idf = {token: math.log((N + 1) / (len(index.get(token, [])) + 1)) + 1 for token in query_tokens}

    # Δημιουργία διανυσμάτων για κάθε έγγραφο
    document_vectors = []
    for doc_id, doc in enumerate(documents):
        doc_vector = [doc['processed_content'].split().count(token) * idf.get(token, 0) for token in query_tokens]
        document_vectors.append(doc_vector)

    # Δημιουργία διανύσματος ερωτήματος
    query_vector = [idf.get(token, 0) for token in query_tokens]

    # Υπολογισμός cosine similarity
    similarities = []
    for doc_id, doc_vector in enumerate(document_vectors):
        dot_product = np.dot(query_vector, doc_vector)
        norm_query = np.linalg.norm(query_vector)
        norm_doc = np.linalg.norm(doc_vector)
        similarity = dot_product / (norm_query * norm_doc) if norm_query and norm_doc else 0
        similarities.append((doc_id, similarity))

    return sorted(similarities, key=lambda x: x[1], reverse=True)

# Επεξεργασία δεδομένων για BM25
def prepare_bm25_data(documents):
    return [doc['processed_content'].split() for doc in documents]

# Αναζήτηση με Okapi BM25
def bm25_search(query, documents):
    tokenized_docs = prepare_bm25_data(documents)
    bm25 = BM25Okapi(tokenized_docs)
    query_tokens = preprocess_query(query)
    print(f"BM25 Query Tokens: {query_tokens}")
    scores = bm25.get_scores(query_tokens)
    return sorted(enumerate(scores), key=lambda x: x[1], reverse=True)

# Διεπαφή αναζήτησης
def search_interface():
    inverted_index, processed_articles = load_data()
    print("Welcome to the Search Engine!")
    print("Enter your query (or type 'exit' to quit):")

    while True:
        query = input("\n> ").strip()
        if query.lower() == 'exit':
            print("Exiting the search engine. Goodbye!")
            break

        print("\nChoose retrieval method:")
        print("1. Boolean Retrieval (AND, OR, NOT)")
        print("2. Vector Space Model (VSM)")
        print("3. Okapi BM25")
        choice = input("\nEnter your choice (1/2/3): ").strip()

        if choice == "1":
            print("\nAvailable Boolean operations: AND, OR, NOT")
            operation = input("Enter Boolean operation: ").strip().upper()
            if operation not in {"AND", "OR", "NOT"}:
                print("Invalid operation. Please try again.")
                continue
            results = boolean_search(query, inverted_index, processed_articles, operation)
            print("\nTop Results:")
            for doc_id in results:
                print(f"- {processed_articles[doc_id]['title']}")
        elif choice == "2":
            results = vsm_search(query, inverted_index, processed_articles)
            print("\nTop Results:")
            for doc_id, score in results[:5]:  # Display top 5
                print(f"- {processed_articles[doc_id]['title']} (Score: {score:.4f})")
        elif choice == "3":
            results = bm25_search(query, processed_articles)
            print("\nTop Results:")
            for doc_id, score in results[:5]:  # Display top 5
                print(f"- {processed_articles[doc_id]['title']} (Score: {score:.4f})")
        else:
            print("Invalid choice. Please try again.")
            continue

# Εκκίνηση διεπαφής
if __name__ == "__main__":
    search_interface()
